In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pybaseball import schedule_and_record
from pybaseball import cache
from pybaseball import batting_stats
from pybaseball import statcast

cache.enable()

In [2]:
data = statcast("2019-06-24", "2019-06-24")

This is a large query, it may take a moment to complete


100%|██████████| 1/1 [00:07<00:00,  7.26s/it]


In [3]:
data

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
965,SL,2019-06-24,86.4,0.3,6.34,"Chapman, Aroldis",520471,547973,field_out,hit_into_play,...,10,8,10,8,10,Standard,Standard,274,0.011,-0.081
1011,FF,2019-06-24,101.3,0.59,6.41,"Chapman, Aroldis",520471,547973,NaN,foul,...,10,8,10,8,10,Standard,Standard,158,0.0,0.0
1042,SL,2019-06-24,89.3,0.32,6.34,"Chapman, Aroldis",520471,547973,NaN,foul,...,10,8,10,8,10,Standard,Standard,251,0.0,0.0
1066,SL,2019-06-24,88.4,0.4,6.36,"Chapman, Aroldis",520471,547973,NaN,foul,...,10,8,10,8,10,Standard,Standard,264,0.0,0.0
1097,SI,2019-06-24,101.8,0.3,6.3,"Chapman, Aroldis",520471,547973,NaN,foul,...,10,8,10,8,10,Standard,Standard,133,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,FF,2019-06-24,89.6,-1.08,6.33,"Greinke, Zack",657077,425844,NaN,called_strike,...,0,0,0,0,0,Infield shift,Standard,188,0.0,-0.067
1910,CH,2019-06-24,87.4,-1.1,6.35,"Greinke, Zack",592626,425844,single,hit_into_play,...,0,0,0,0,0,Infield shift,Standard,250,-0.036,0.367
1939,FF,2019-06-24,91.1,-1.06,6.24,"Greinke, Zack",592626,425844,NaN,ball,...,0,0,0,0,0,Infield shift,Standard,181,0.0,0.054
2047,FF,2019-06-24,90.3,-1.04,6.38,"Greinke, Zack",592626,425844,NaN,foul,...,0,0,0,0,0,Infield shift,Standard,202,0.0,-0.05


In [4]:
data.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

In [5]:
from pybaseball import batting_stats_range

# retrieve all players' batting stats for the month of May, 2017
data = batting_stats_range("2017-05-01", "2017-05-28")

In [6]:
data

,Name,Age,#days,Lev,Tm,G,PA,AB,R,H,...,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,mlbID
1,Jos\xc3\xa9 Abreu,30,2123,Maj-AL,Chicago,26,117,110,22,35,...,0,0,5,0,0,0.318,0.359,0.591,0.950,547989
2,Lane Adams,27,2144,Maj-NL,Atlanta,6,6,6,0,2,...,0,0,1,1,0,0.333,0.333,0.333,0.667,572669
3,Matt Adams,28,2123,Maj-NL,"Atlanta,St. Louis",19,48,47,7,13,...,0,0,2,0,0,0.277,0.292,0.553,0.845,571431
4,Jim Adduci,32,2141,Maj-AL,Detroit,7,27,24,3,6,...,0,0,0,0,0,0.250,0.333,0.375,0.708,451192
5,Tim Adleman,29,2125,Maj-NL,Cincinnati,4,9,9,0,0,...,0,0,0,0,0,0.000,0.000,0.000,0.000,534947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,Rob Zastryzny,25,2145,Maj-NL,Chicago,1,2,2,0,0,...,0,0,0,0,0,0.000,0.000,0.000,0.000,642239
645,Bradley Zimmer,24,2123,Maj-AL,Cleveland,11,32,27,4,8,...,0,0,0,2,1,0.296,0.406,0.482,0.888,605548
646,Ryan Zimmerman,32,2123,Maj-NL,Washington,22,89,82,9,25,...,0,1,5,0,0,0.305,0.348,0.476,0.824,475582
647,Ben Zobrist,36,2125,Maj-NL,Chicago,20,76,68,5,21,...,0,0,0,0,0,0.309,0.382,0.529,0.911,450314


In [7]:
data.columns

Index(['Name', 'Age', '#days', 'Lev', 'Tm', 'G', 'PA', 'AB', 'R', 'H', '2B',
       '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF', 'GDP', 'SB',
       'CS', 'BA', 'OBP', 'SLG', 'OPS', 'mlbID'],
      dtype='object')

In [19]:
# initialize an empty dataframe
df = pd.DataFrame()

# Loop through all the dates (as strings) in the month of May, 2017
for date in pd.date_range("2015-01-01", "2022-12-31").strftime("%Y-%m-%d"):
    # Retrieve the batting stats for that day
    data = batting_stats_range(date, date)
    # Append the data to the dataframe
    df = df.concat(data)

/tmp/ipykernel_678/3289479983.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data)
/tmp/ipykernel_678/3289479983.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data)


In [18]:
print(pd.date_range("2017-05-01", "2017-05-02").strftime("%Y-%m-%d"))


Index(['2017-05-01', '2017-05-02'], dtype='object')


In [20]:
df

,Name,Age,#days,Lev,Date,Tm,,Opp,G,PA,...,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,mlbID
1,Jos\xc3\xa9 Abreu,30,2150,Maj-AL,"May 1, 2017",Chicago,@,Kansas City,1,4,...,0,0,0,0,0,0.000,0.000,0.000,0.000,547989
2,Lane Adams,27,2150,Maj-NL,"May 1, 2017",Atlanta,,New York,1,1,...,0,0,0,0,0,0.000,0.000,0.000,0.000,572669
3,Matt Adams,28,2150,Maj-NL,"May 1, 2017",St. Louis,,Milwaukee,1,1,...,0,0,0,0,0,1.000,1.000,1.000,2.000,571431
4,Jim Adduci,32,2150,Maj-AL,"May 1, 2017",Detroit,,Cleveland,1,4,...,0,0,0,0,0,0.000,0.000,0.000,0.000,451192
5,Jes\xc3\xbas Aguilar,27,2150,Maj-NL,"May 1, 2017",Milwaukee,@,St. Louis,1,4,...,0,0,1,0,0,0.000,0.000,0.000,0.000,542583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,Alex Wood,26,2149,Maj-NL,"May 2, 2017",Los Angeles,,San Francisco,1,2,...,0,0,0,0,0,0.500,0.500,0.500,1.000,622072
329,Christian Yelich,25,2149,Maj-NL,"May 2, 2017",Miami,,Tampa Bay,1,4,...,0,0,0,0,0,0.000,0.000,0.000,0.000,592885
330,Ryan Zimmerman,32,2149,Maj-NL,"May 2, 2017",Washington,,Arizona,1,5,...,0,0,1,0,0,0.400,0.400,0.600,1.000,475582
331,Ben Zobrist,36,2149,Maj-NL,"May 2, 2017",Chicago,,Philadelphia,1,4,...,0,0,0,0,0,0.333,0.500,0.667,1.167,450314


In [21]:
# print only where ben zobrist is the name
df[df['Name'] == 'Ben Zobrist']

,Name,Age,#days,Lev,Date,Tm,,Opp,G,PA,...,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,mlbID
240,Ben Zobrist,36,2150,Maj-NL,"May 1, 2017",Chicago,,Philadelphia,1,4,...,0,0,0,0,0,0.250,0.25,0.250,0.500,450314
331,Ben Zobrist,36,2149,Maj-NL,"May 2, 2017",Chicago,,Philadelphia,1,4,...,0,0,0,0,0,0.333,0.50,0.667,1.167,450314
